## Feature Engineering

How can we predict clean sheets and goals? (Follow up from feature_analysis)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import random
import pingouin as pg
import glob
import re
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
# Current gameweek 
gameweek = 16

## Collect available player data

In [3]:
# Initialize an empty list to store all individual, player gameweek data 
all_player_sep = []

# Loop through each gameweek
for i in range(1, gameweek + 1):  # Adjusting the range to start from 1 to gameweek
    # Read the CSV for the current gameweek
    x = pd.read_csv(rf'C:\Users\thoma\Code\Projects\Fantasy-Premier-League\Data\Players\Seperate_GW\GW_{i}.csv')
    
    # Append the current gameweek data to the list
    all_player_sep.append(x)

# Concatenate all dataframes in the list into a single dataframe
player_data = pd.concat(all_player_sep, axis=0, ignore_index=True)

# Drop unnamed column
player_data = player_data.drop(columns = ['Unnamed: 0'])

# Sort dataset correctly IMPORTANT
player_data = player_data.sort_values(by= ['Player ID','Gameweek'])

## Fixture Difficulty Rating (updated version)

In [4]:
# Read the difficulty data
difficulty = pd.read_csv(r'C:\Users\thoma\Code\Projects\Fantasy-Premier-League\Data\Fixtures\Difficulty_ratings\FD_Improved\Current_FD_Improved.csv', index_col=0)

# Create a mapping dictionary
mapping = difficulty.set_index(['Opponent', 'Position'])['FD_combined'].to_dict()

# Apply the mapping to a new column in player_data
player_data['FD_combined'] = player_data.apply(
    lambda row: mapping.get((row['Opponent'], row['Position']), None), axis=1
)

## Team data

In [5]:
# Specify the path to the files
attack = glob.glob(r'C:\Users\thoma\Code\Projects\Fantasy-Premier-League\Data\Team\Seperate_GW\Attacking\*.csv')
defense = glob.glob(r'C:\Users\thoma\Code\Projects\Fantasy-Premier-League\Data\Team\Seperate_GW\Defensive\*.csv')

# Define a function to extract the week number from the filename
def extract_week_number(filename):
    match = re.search(r'GW_(\d+)', filename)
    return int(match.group(1)) if match else None

# Read each attacking file and add the 'Week' column
att_weekly_data = pd.concat(
    [pd.read_csv(file).assign(Week=extract_week_number(file)) for file in attack],
    ignore_index=True
)

# Read each defensive file and add the 'Week' column
def_weekly_data = pd.concat(
    [pd.read_csv(file).assign(Week=extract_week_number(file)) for file in defense],
    ignore_index=True
)
# Remove 'VS' team from defensive data
def_weekly_data['Team'] = def_weekly_data['Team'].str[3:]

# Choose columns data 
columns_new = ['Team','Week', 'Playing TimeMP', 'Possession','PerformanceGls','PerformanceAst','ExpectedxG','ExpectedxAG',
               'Per 90 MinutesGls','Per 90 MinutesAst','Per 90 MinutesxG','Per 90 MinutesxAG']

# Attacking data
attacking_data = pd.DataFrame(att_weekly_data[columns_new]).sort_values(by = 'Week')

# # Defensive data
defensive_data = pd.DataFrame(def_weekly_data[columns_new]).sort_values(by = 'Week')

# Collect fixture list
fixtures = pd.read_csv(r'C:\Users\thoma\Code\Projects\Fantasy-Premier-League\Data\Fixtures\Schedule\Fixtures_alt_names.csv')

# Create function to collect fixture data
def fixture_data(team, fixtures, gameweek):
    
    # Create empty list of fixtures
    fix_data = []
    # Iterate over each row of the fixtures DataFrame
    for index, row in fixtures.iterrows():
        # Check if the row's team matches the input team
        if row['Team'] == team:
            # Loop through the columns corresponding to gameweeks
            for col in fixtures.columns[1:gameweek + 1]:
                if '(H)' or '(A)' in row[col]:  # Check row has fixture information
                    fix_data.append([col, row[col]])

    # Return the collected home data
    return fix_data

# Get games
games = []

# List of unique teams 
teams = attacking_data['Team'].unique()

# For all teams
for team in teams:
    # Get fixture information
    fix_data = fixture_data(team, fixtures, gameweek)  # Fetch data for the team
    for info in fix_data:
        # You can extract relevant information from 'game', like opponent, week, etc.
        games.append([info[0], team, info[1]])

# Creating DataFrame for all teams fixture list
fix = pd.DataFrame(games, columns=['Week', 'Team', 'Opponent'])

# Remove 'GW' from the 'Week' string and convert it to an integer
fix['Week'] = fix['Week'].str[2:].astype(int)

# Define columns
cols = ['Team', 'Week', 'Possession', 'PerformanceGls',
       'PerformanceAst', 'ExpectedxG', 'ExpectedxAG', 'Per 90 MinutesGls',
       'Per 90 MinutesAst', 'Per 90 MinutesxG', 'Per 90 MinutesxAG']

# Get attacking and defensive data
attacking = attacking_data[cols]
defensive = defensive_data[cols]

# Merge attacking and defensive data for each team for each gameweek
team_attack = fix.merge(attacking, on=['Week', 'Team'])
team_defense = fix.merge(defensive, on=['Week', 'Team'])

# Rename team_names to align with player_data
# Define a dictionary of old team names as keys and new names as values
name_changes = {
    "Nott'ham Forest": 'Nottingham Forest',
    'Manchester Utd': 'Man Utd',
    'Manchester City': 'Man City',
    'Newcastle Utd': 'Newcastle',
    'Leicester City': 'Leicester',
    'Ipswich Town': 'Ipswich',
    'Tottenham': 'Spurs',
    
}
# Replace the team names using the dictionary
team_attack['Team'] = team_attack['Team'].replace(name_changes)
team_defense['Team'] = team_defense['Team'].replace(name_changes)

# Rename team columns
team_defense.rename(columns=lambda col: f"{col} against", inplace=True)
team_defense.rename(columns={'Week against': 'Week', 'Team against': 'Team', 'Opponent against': 'Opponent'}, inplace=True)

# Merge the playerdata with attacking, and then defensive team information
merged_df = pd.merge(player_data, team_attack, on=['Team', 'Opponent'], how='left')
player_d = pd.merge(merged_df, team_defense, on=['Team', 'Opponent'], how='left')

# Drop uneeded columns
player_data = player_d.drop(columns = ['Week_x', 'Week_y', 'KO_time'])

# Collect columns that are averages of team performance (per_90)
Per_90 = player_data[['Player ID', 'Gameweek','Per 90 MinutesxG', 'Per 90 MinutesGls', 'Per 90 MinutesxG against','Per 90 MinutesGls against']]

# Filter on latest GW possible to get most accurate average value
Per_90 = Per_90[Per_90['Gameweek'] == 8]

# Merge the data on 'Player ID'
complete = player_data.merge(Per_90, on='Player ID', how='left', suffixes=('_postGW8', '_preGW8'))

# Replace NaN values in POST_8 variables with average values from 'Per 90 MinutesxG_team'
complete['PerformanceGls'] = complete['PerformanceGls'].fillna(complete['Per 90 MinutesGls_preGW8'])
complete['ExpectedxG'] = complete['ExpectedxG'].fillna(complete['Per 90 MinutesxG_preGW8'])
complete['PerformanceGls against'] = complete['PerformanceGls against'].fillna(complete['Per 90 MinutesGls against_preGW8'])
complete['ExpectedxG against'] = complete['ExpectedxG against'].fillna(complete['Per 90 MinutesxG against_preGW8'])

# Rename column
complete = complete.rename(columns={'Gameweek_postGW8': 'Gameweek',
                                    'PerformanceGls': 'Team_gls',
                                    'ExpectedxG': 'TeamxG',
                                    'PerformanceGls against': 'Team_gls_against',
                                    'ExpectedxG against': 'TeamxG_against',
                                    })

columns_to_keep = ['Player ID', 'Name', 'Last_Name', 'Team', 'Position', 'Cost_Today',
       'GW Points', 'Minutes', 'Goals', 'Assists', 'Clean Sheets',
       'Goals Conceded', 'Penalties Saved', 'Penalties Missed', 'YC', 'RC',
       'Saves', 'Total Bonus Points', 'Total BPS', 'Influence', 'Creativity',
       'Threat', 'ICT Index', 'xG', 'xA', 'xGi', 'xGc', 'Transfers In GW',
       'Transfers Out GW', 'Gameweek', 'Opponent', 'FD_combined',
       'Team_gls', 'TeamxG', 'Team_gls_against', 'TeamxG_against']

# New player data with columns
player_data = complete[columns_to_keep]

# Sort dataset correctly IMPORTANT
player_data = player_data.sort_values(by= ['Player ID','Gameweek'])

# Feature_engineering

# Fixture Difficulty Difference

In [6]:
# Opponent difficulty ( = same as FD_combined)
player_data['Opponent_Difficulty'] = player_data['FD_combined'].rename(inplace= True)

# Initialize a list to store the results
player_difficulty = []

# Iterate through each player and consecutive gameweek
for _, row in player_data.iterrows():
    team = row['Team']  # Get the player's team
    player_position = row['Position']  # Get the player's position
    opponent_info = row['Opponent']  # Get the player's opponent info
    
    # Filter difficulty list for the player's team
    difficulty_filtered = difficulty[difficulty['Team'] == team]
    
    # Player played at home
    if "(H)" in opponent_info:
        # Filter by (A), as this is from opponent perspective
        player = difficulty_filtered[difficulty_filtered['Opponent'].str.contains(r"\(A\)")]
    # Player played away
    elif "(A)" in opponent_info:
        # Filter by (H), as this is from opponent perspective
        player = difficulty_filtered[difficulty_filtered['Opponent'].str.contains(r"\(H\)")]
    else:
        continue

    # Determine difficulty based on player position
    if player_position in ['MID', 'FWD']:
        difficulty_player = player[player['Position'] == 'DEF']
    elif player_position in ['GK', 'DEF']:
        difficulty_player = player[player['Position'] == 'FWD']
    else:
        continue  # Skip if position is not recognized

    # Collect the value that remains after filtering away irrelvant information
    score = difficulty_player['FD_combined'].sum()

    # Append the result for this player
    player_difficulty.append({
        'Player ID': row['Player ID'],
        'Opponent': opponent_info,
        'Player_Difficulty': score
    })

# Convert to DataFrame, excluding the Difficulty DataFrame
player_difficulty_summary = pd.DataFrame(player_difficulty)

# Merge back with original dataframe 
player_data = player_data.merge(player_difficulty_summary, on = ['Player ID', 'Opponent'])

# Create difficulty difference
player_data['FD_Difference'] = player_data['Player_Difficulty'] - player_data['Opponent_Difficulty']

## Feature Engineering: Rolling averages

In [7]:
number_of_games = 4  # Define the window size

# Apply rolling mean for "Form" (GW Points) (excluding current gameweek)
player_data["Form"] = (
    player_data.groupby("Player ID")["GW Points"]
    .transform(lambda x: x.shift(1).rolling(window=number_of_games).mean().round(3))
)

# Apply rolling mean for "Form_xG" (excluding current gameweek)
player_data["Form_player_xG"] = (
    player_data.groupby("Player ID")["xG"]
    .transform(lambda x: x.shift(1).rolling(window=number_of_games).mean().round(3))
)

# Apply rolling mean for "Form_xGc" (excluding current gameweek)
player_data["Form_player_xGc"] = (
    player_data.groupby("Player ID")["xGc"]
    .transform(lambda x: x.shift(1).rolling(window=number_of_games).mean().round(3))
)

# Apply rolling mean for "Team xG" (excluding current gameweek)
player_data["Form_Team_xG"] = (
    player_data.groupby("Player ID")["TeamxG"]
    .transform(lambda x: x.shift(1).rolling(window=number_of_games).mean().round(3))
)

# Apply rolling mean for "Team xG against" (excluding current gameweek)
player_data["Form_TeamxG_against"] = (
    player_data.groupby("Player ID")["TeamxG_against"]
    .transform(lambda x: x.shift(1).rolling(window=number_of_games).mean().round(3))
)

# Apply roling mean for 'Team performance goals' (excluding current gameweek
player_data["Form_Team_gls"] = (
    player_data.groupby("Player ID")["Team_gls"]
    .transform(lambda x: x.shift(1).rolling(window=number_of_games).mean().round(3))
)

# Apply roling mean for 'Team performance goals' (excluding current gameweek
player_data["Form_Team_gls_against"] = (
    player_data.groupby("Player ID")["Team_gls_against"]
    .transform(lambda x: x.shift(1).rolling(window=number_of_games).mean().round(3))
)

In [8]:
# Remove players who play less than 61 minutes in a game (i.e. they do not recieve their 2 points minimum for playoing this amount)
player_data = player_data[player_data['Minutes'] > 60].copy()

# Moderators

A moderator effect occurs, when there is the relationship between two variables changes when you include another variable. 

For example, is there a different relationship between Clean Sheets and Fixture Difficulty for teams who have recently been giving away many more scoring chances than others (xG against).

To assess for moderation effect, we simply need to create an interaction variable (which is one variable multiplied by the other). 

## Multicollinearity

This occurs when 2 or more predictors are highly correlated with each other. It is problematic, because parameters become interchangeable, as the model cannot discriminate between each predictor. 

It makes interpreting the model much more difficult, due to increased 'noise'. 

Noise refers to irrelevant information that does not contribute to the predictive relationship.

In the context of multicollinearity, noise can arise when redundant predictors inflate the variability in coefficient estimates, making it difficult to discern the true relationship.

How to check for multicollinearity?

- Use r^2 (if > 80% you're in trouble)
- Use Variance Inflation Factor (VIF) = 1/1 - r^2 (1 < 5 indicates moderate; > 5 indicates severe)

How to control for multicollinearity?

1) Remove one of the variables
2) Create a composite variable (using principal component analysis)
3) Centre (standardize) variables: subtract each row value by the column mean into a new columns 
    e.g. [player_data['GW_Points_c'] = player_data['GW Points'] - player_data['GW Points'].mean()]

In [9]:
# Min/Max scaler 
scaler = MinMaxScaler(feature_range=(2, 5))

# Normalise FD_Difference
player_data['FD_Difference_norm'] = scaler.fit_transform(player_data['FD_Difference'].values.reshape(-1, 1))

# Normalise PlayerxG form
player_data['Form_player_xG_norm'] = scaler.fit_transform(player_data['Form_player_xG'].values.reshape(-1,1))

# Normalise TeamxG_against
player_data['Form_TeamxG_against_norm'] = scaler.fit_transform(player_data['Form_TeamxG_against'].values.reshape(-1,1))

# Normalise Form
player_data['Form_norm'] = scaler.fit_transform(player_data['Form'].values.reshape(-1,1))

In [10]:
# Non-normalized variables

# Creating interaction variable (for defensive players - based on best variables)
player_data['FD_xGA_interaction'] = player_data['FD_Difference'] / player_data['Form_TeamxG_against']

# Creating interaction variable (for attacking players - based on best variables)
player_data['FD_Diff_xG_interaction'] = player_data['FD_Difference'] * player_data['Form_player_xG']

# Creating add on interaction variable (for attacking players - based on best variables)
player_data['Form_FD_xG_interaction'] = player_data['FD_Diff_xG_interaction'] * player_data['Form']

# Creating interaction variable (for attacking players - based on best variables)
player_data['Form_FD_Diff_interaction'] = player_data['FD_Difference'] * player_data['Form']

In [11]:
# Normalised variables

# Creating interaction variable (for defensive players - based on best variables)
player_data['FD_xGA_interaction_norm'] = player_data['FD_Difference_norm'] / player_data['Form_TeamxG_against_norm']

# Creating interaction variable (for attacking players - based on best variables)
player_data['FD_Diff_xG_interaction_norm'] = player_data['FD_Difference_norm'] * player_data['Form_player_xG_norm']

# Creating add on interaction variable (for attacking players - based on best variables)
player_data['Form_FD_xG_interaction_norm'] = player_data['FD_Diff_xG_interaction_norm'] * player_data['Form_norm']

# Creating interaction variable (for attacking players - based on best variables)
player_data['Form_FD_Diff_interaction_norm'] = player_data['FD_Difference_norm'] * player_data['Form_norm']

In [12]:
# Form/Fixture Difficulty
player_data['Form_FD'] = player_data['Form'] / player_data['FD_combined']

# xG/Fixture Difficulty
player_data['Form_player_xG_FD'] = player_data['Form_player_xG'] / player_data['FD_combined']

# xGc/Fixture Difficulty
player_data['Form_player_xGc_FD'] = player_data['Form_player_xGc'] / player_data['FD_combined']

In [13]:
# Columns for correlations
columns = player_data.columns.drop(['Player ID', 'Name', 'Last_Name', 'Team', 'Position', 'Cost_Today', 'Opponent'])

# Defensive and Forward players
attackers = player_data[player_data['Position'].isin(['MID', 'FWD'])].copy()
defenders = player_data[player_data['Position'].isin(['GK', 'DEF'])].copy()

# Sort the correlation matrix for defenders
corr = attackers[columns].corr().sort_values(by='Goals', ascending=False)
corr.head(20)

,GW Points,Minutes,Goals,Assists,Clean Sheets,Goals Conceded,Penalties Saved,Penalties Missed,YC,RC,...,FD_Diff_xG_interaction,Form_FD_xG_interaction,Form_FD_Diff_interaction,FD_xGA_interaction_norm,FD_Diff_xG_interaction_norm,Form_FD_xG_interaction_norm,Form_FD_Diff_interaction_norm,Form_FD,Form_player_xG_FD,Form_player_xGc_FD
Goals,0.866981,0.061582,1.000000,0.081788,0.000733,0.014190,NaN,-0.003554,-0.026781,0.000642,...,0.238223,0.230404,0.214472,0.121071,0.258850,0.252284,0.241214,0.216463,0.227178,0.099788
GW Points,1.000000,0.076673,0.866981,0.500441,0.110913,-0.085481,NaN,-0.033660,-0.143708,-0.060174,...,0.251194,0.240873,0.241983,0.169801,0.279964,0.267771,0.275343,0.221911,0.215563,0.088449
Influence,0.892403,0.195027,0.843411,0.387138,0.020694,0.002951,NaN,0.000900,-0.034196,-0.020863,...,0.242896,0.220854,0.235680,0.165562,0.236493,0.217244,0.256783,0.186671,0.140193,0.131074
Total BPS,0.895035,0.161562,0.799374,0.440958,0.031930,-0.001496,NaN,-0.030503,-0.108651,-0.044025,...,0.256076,0.226866,0.262355,0.205317,0.255110,0.223723,0.279250,0.189493,0.137146,0.131642
Total Bonus Points,0.855276,0.114041,0.757734,0.299973,-0.007196,-0.052159,NaN,-0.013886,-0.044608,-0.022744,...,0.214439,0.218843,0.196497,0.118670,0.233939,0.229160,0.219789,0.188673,0.196685,0.093462
ICT Index,0.741330,0.195779,0.667471,0.363265,0.045803,-0.036562,NaN,0.021773,-0.050951,-0.018334,...,0.298144,0.270023,0.313042,0.268141,0.370928,0.347947,0.379215,0.299989,0.265826,0.161417
xG,0.544568,0.052129,0.629151,0.102656,0.009774,0.000064,NaN,0.136538,-0.061022,-0.001598,...,0.298052,0.275307,0.264152,0.206629,0.379450,0.360538,0.324281,0.290714,0.359168,0.138673
Threat,0.548948,0.083266,0.589124,0.134344,0.038061,-0.042722,NaN,0.042562,-0.063141,0.006916,...,0.306714,0.284138,0.281901,0.237911,0.431698,0.416893,0.375405,0.333971,0.401877,0.139674
xGi,0.576477,0.096249,0.572179,0.249435,0.004082,-0.001993,NaN,0.120063,-0.051333,-0.013786,...,0.318740,0.293579,0.323186,0.265276,0.406178,0.384095,0.386075,0.323101,0.335135,0.154665
FD_Diff_xG_interaction_norm,0.279964,0.036809,0.258850,0.140076,-0.014704,-0.049876,NaN,-0.018125,-0.088310,-0.019316,...,0.733424,0.670579,0.752975,0.669893,1.000000,0.925960,0.886649,0.650850,0.789877,0.270088


## Mediation (partial correlation/confounders)

Assesses the unique relationship between two variables, when controlling for another variable (also known as a mediator or confounder). 

The confounding variable may suppress the interation or real relationship between the other variables. The null hypothesis is that after controlling for the third variable (or confounder), there is no relationship between a and b.

For example, is there a relationship between coffee consumption and productivity? One potential confounder that definately effects both of these variables is sleep quality. Lower sleep means increased coffee consumption and lower productivity. Better sleep means reduced coffee consumption and higher productivity. If everyone slept the same amount, would there be a relationship (in terms of co-variance/correlation) between coffee and productivity?

Equation = 'Relationship between a and b, while controlling for variable c.'
> rab.c = rab - rac-rbc/ sqrt(1-r^2ac)sqrt(1-r^2bc).
Relationship between a and b, while controlling for variable c. 

The method of partial correlation is to use the residuals of other variables. 

Regress Y onto X: Y is your target variable, X is your predictor. You determine the part of Y that can be explained by X. A better model has lower residuals. Either way, the residuals (Y') represent the portion of Y that is unrelated to X.

Regress Z onto X: Z is your target variable, X is your predictor. You determine the part of Z that can be explained by X. A better model has lower residuals. Either way, the residuals (Z') represent the portion of Z that is not explained by X.

Correlation between residuals of (Y' and Z') reflects the relationship between these two without the influence of X.

In [14]:
partial_corr_results = pg.partial_corr(attackers, x = 'GW Points', y = 'ICT Index', covar = ['Threat', 'Creativity'])
# Most of the ICT index is actually made up of 'Influence'. Controlling for threat and creativity havs little effect on
# the direct relationship between the two. Threat and Creativity are not significant confounders 
# between ICT and Influence for attackers.

In [15]:
partial_corr_results = pg.partial_corr(attackers, x = 'Goals', y = 'xG', covar = ['Threat'])

# Threat represents a fairly strong confounder of the relationship between goals and xG. The correlation
# drops from 0.6 to 0.3.

## Key Conclusions

Our feature engineering process was built from our feature analysis using linear regression. We undercovered that defensives players were getting most points from clean sheets and attacking players
were getting most points for scoring goals. To keep the model simple, we split by position and created two distinct features that share a good amount of variance with goals scored and clean sheets. 

Attacking players: Predicting goals 

- Form * Fixture_Difficulty_Difference * Form_player_xG (both normalised between 2-5) (shares 25% variance)

Defending players: Predicting clean sheets

- Fixture_Difficulty_Difference / Form_Team_xG_against (shares 35% variance)